In [34]:
import re
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import optuna as opt

In [35]:
db = pd.read_excel("/Users/sigurdskyrud/OneDrive/Dokumenter/Andre Dokumenter/Billettkontroll data.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Linje     37 non-null     object        
 1   Vogn      37 non-null     object        
 2   Dato      37 non-null     datetime64[ns]
 3   Klokke    37 non-null     object        
 4   Fra       37 non-null     object        
 5   Til       37 non-null     object        
 6   Fullt?    37 non-null     object        
 7   Sjekket?  37 non-null     object        
 8   Merknad   3 non-null      object        
dtypes: datetime64[ns](1), object(8)
memory usage: 2.7+ KB


In [46]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Linje     37 non-null     int8          
 1   Vogn      37 non-null     int8          
 2   Dato      37 non-null     datetime64[ns]
 3   Klokke    37 non-null     object        
 4   Fra       37 non-null     int8          
 5   Til       37 non-null     int8          
 6   Fullt?    37 non-null     int8          
 7   Sjekket?  37 non-null     int8          
 8   Merknad   3 non-null      object        
dtypes: datetime64[ns](1), int8(6), object(2)
memory usage: 1.2+ KB


In [36]:
db["Fullt?"] = pd.Categorical(db["Fullt?"]).codes
db["Sjekket?"] = pd.Categorical(db["Sjekket?"]).codes
db["Vogn"] = pd.Categorical(db["Vogn"]).codes
db["Linje"] = pd.Categorical(db["Linje"]).codes
db["Fra"] = pd.Categorical(db["Fra"]).codes
db["Til"] = pd.Categorical(db["Til"]).codes

In [47]:
def clockTofloat(input: str) -> float:
    input = re.sub(r'[:]', )

In [45]:
db

,Linje,Vogn,Dato,Klokke,Fra,Til,Fullt?,Sjekket?,Merknad
0,1,2,2022-09-25,11:30:00,0,2,1,1,NaN
1,3,0,2022-09-25,21:59:00,2,0,0,0,NaN
2,1,2,2022-09-26,09:30:00,0,1,0,1,NaN
3,1,0,2022-09-26,13:48:00,1,0,0,1,NaN
4,4,2,2022-09-27,06:00:00,0,2,1,0,NaN
5,4,0,2022-09-27,12:13:00,2,3,2,0,NaN
6,1,0,2022-09-27,14:54:00,3,0,0,0,Storkontroll
7,1,1,2022-09-28,09:30:00,0,3,1,1,NaN
8,0,4,2022-09-28,12:48:00,1,0,0,1,NaN
9,3,2,2022-09-29,07:51:00,0,1,0,1,"Sjekket, ble glemt"
